In [1]:
utils_path = '/Users/aaronwu/Documents/misc/tmp/selenium/'
import sys
sys.path.append(utils_path)

from selenium_helper import *
from moonboard_helper import *

In [2]:
from selenium import webdriver
import pickle
import os
import pdb
import time

In [3]:
def check_fail(driver):
    elems = find_element(driver, 'span', 'class', 'field-validation-error', num_tries=1)
    if elems!=None:
        return False
    return True

def check_url(driver, url):
    driver.get(url)
    check = check_fail(driver)
    return check

In [4]:
def get_problem_meta(driver):
    problem = None
    divs = driver.find_elements_by_tag_name('div')
    for div in divs:
        if div.get_attribute('class')=='problem-inner view':
            problem = div
            break
    if problem==None:
        return None
    meta = {}
    h3 = problem.find_elements_by_tag_name('h3')[0]
    meta['name'] = h3.text
    meta['info'] = [p.text for p in problem.find_elements_by_tag_name('p')]
    stars = [star.get_attribute('src') for star in problem.find_elements_by_tag_name('img')]
    stars = [star for star in stars if 'star' in star]
    meta['num_empty'] = len([star for star in stars if 'empty' in star])
    meta['num_stars'] = len(stars) - meta['num_empty']
    return meta

def click_next_page(driver, current_page):
    next_page=current_page+1
    pager = None
    clicked = False
    spans = driver.find_elements_by_tag_name('span')
    for s in spans:
        if s.get_attribute('class')=='k-link k-pager-nav':
            pager=s
    if pager==None:
        return clicked
    pager.click()
    items = driver.find_elements_by_tag_name('a')
    for i in items:
        if i.get_attribute('data-page')==str(next_page):
            i.click()
            return True
    return False

def get_repeats_data(driver):
    num_tries = 20
    repeats_data = []
    attr_dict={'class':'repeats'}
    current_page=1
    while True:
        repeats_add=None
        for x in range(num_tries):
            try:
                repeats = get_elem_set(driver, 'div', attr_dict)
                for repeat in repeats:
                    h3_item = repeat.find_elements_by_tag_name('h3')[0]
                    user_name = h3_item.text
                    uid = h3_item.find_elements_by_tag_name('a')[0].get_attribute('href').split('/Account/Profile/')[1]
                    repeats_add = [r.text for r in repeat.find_elements_by_tag_name('p')] + [user_name, uid]
                    repeats_data.append(repeats_add)
                break
            except:
                time.sleep(.5)
                if x==num_tries-1:
                    pdb.set_trace()
                continue
            if repeats_add==None:
                break
            if len(remove_duplicates(repeats_add))<6:
                break
        if repeats_add==None:
            return None
        for x in range(num_tries):
            try:
                clicked = click_next_page(driver, current_page)
                break
            except:
                time.sleep(.5)
                continue
        if not clicked:
            break
        current_page+=1
    repeats_data = remove_duplicates(repeats_data)
    return repeats_data

def remove_duplicates(repeats_data):
    repeats = []
    for r in repeats_data:
        if not r in repeats:
            repeats.append(r)
    return repeats

In [5]:
def search_correct_url(driver, url, uid, start_dev=0, radius=10):
    split_url = url.split(uid)
    uid_int = int(uid)+start_dev
    uid_list = [uid_int]
    for r in range(1,radius+1):
        uid_list.append(uid_int+r)
        uid_list.append(uid_int-r)
    for i in uid_list:
        new_url = split_url[0]+str(i)+split_url[1]
        if check_url(driver, new_url):
            return i, new_url
    return -1, ''

def correct_uids(driver, problems_dict, found_dict={}, unfound_dict={}, checked_ids=[]):
    uids = sorted(problems_dict.keys())
    for i,uid in enumerate(uids):
        print('\n')
        print(str(i)+'/'+str(len(uids)))
        if uid in checked_ids:
            continue
        new_uid, new_url = search_correct_url(driver, problems_dict[uid]['url'], uid)
        if new_uid==-1:
            unfound_dict[uid] = problems_dict[uid]
        else:
            found_dict[new_uid] = problems_dict[uid]
            found_dict[new_uid]['url'] = new_url
    return found_dict, unfound_dict, checked_ids

In [6]:
def check_repeats(repeats_data, meta_data):
    if ' climber' in meta_data['info'][1]:
        num_repeats1 = int(meta_data['info'][1].split(' climber')[0])
    else:
        num_repeats1 = 0
    repeats_data = [r for r in repeats_data if r[0]!='Project']
    repeat_unique = list(set(['-'.join(r[-2:]).lower() for r in repeats_data]))
    repeat_users = list(set([r[-2].lower() for r in repeats_data]))
    num_repeats2 = len(repeat_unique)
    if len(repeats_data)>0:
        if repeats_data[0][0]=='':
            return False
    if (num_repeats1!=num_repeats2) and (num_repeats1!=num_repeats2-1):
        return False
    return True

def debug_repeats(repeats_data, meta_data):
    if ' climber' in meta_data['info'][1]:
        num_repeats1 = int(meta_data['info'][1].split(' climber')[0])
    else:
        num_repeats1 = 0
    repeats_data = [r for r in repeats_data if r[0]!='Project']
    repeat_unique = list(set(['-'.join(r[-2:]).lower() for r in repeats_data]))
    repeat_users = list(set([r[-2].lower() for r in repeats_data]))
    num_repeats2 = len(repeat_unique)
    if len(repeats_data)>0:
        if repeats_data[0][0]=='':
            pdb.set_trace()
    if (num_repeats1!=num_repeats2) and (num_repeats1!=num_repeats2-1):
        pdb.set_trace()
    return

In [7]:
parent_path = '/Users/aaronwu/Documents/misc/tmp/selenium/'
executable_path = parent_path + 'chromedriver'
error_path = '/Users/aaronwu/Documents/misc/MoonBoard/data/error/'
error_name = 'invalid_urls.pickle'

out_name1 = 'corrected_urls.pickle'
out_name2 = 'uncorrected_urls.pickle'

url = 'https://www.moonboard.com/'
username = 'aaronlwu257@hotmail.com'
password = 'temppass1992'

restart = False

In [8]:
error_dict = pickle.load(open(error_path+error_name,'rb'))

In [9]:
driver = load_driver(executable_path=executable_path)

In [10]:
loginMoonBoard(driver, url=url, username=username, password=password)
time.sleep(2)

In [11]:
corrected_repeats_dict = {}
uncorrected_repeats_dict = {}
if os.path.exists(error_path+out_name1) and not restart:
    corrected_repeats_dict = pickle.load(open(error_path+out_name1,'rb'))
if os.path.exists(error_path+out_name2) and not restart:
    uncorrected_repeats_dict = pickle.load(open(error_path+out_name2,'rb'))

keys = list(error_dict['bad_repeats'].keys())
for i,k in enumerate(keys):
    print(str(i)+'/'+str(len(keys)))
    if (k in corrected_repeats_dict) or (k in uncorrected_repeats_dict):
        continue
    if check_url(driver, error_dict['bad_repeats'][k]):
        repeats_data = get_repeats_data(driver)
        if repeats_data==None:
            uncorrected_repeats_dict[k]={'url':error_dict['bad_repeats'][k]}
            pickle.dump(uncorrected_repeats_dict, open(error_path+out_name2,'wb'))
            continue
        meta_data = get_problem_meta(driver)
        match = check_repeats(repeats_data, meta_data)
        if repeats_data!=None:
            corrected_repeats_dict[k]={'url':error_dict['bad_repeats'][k], 'repeats':repeats_data, 'meta':meta_data, 'match':match}
            pickle.dump(corrected_repeats_dict, open(error_path+out_name1,'wb'))
            

0/10870
1/10870
2/10870
3/10870
4/10870
5/10870
6/10870
7/10870
8/10870
9/10870
10/10870
11/10870
12/10870
13/10870
14/10870
15/10870
16/10870
17/10870
18/10870
19/10870
20/10870
21/10870
22/10870
23/10870
24/10870
25/10870
26/10870
27/10870
28/10870
29/10870
30/10870
31/10870
32/10870
33/10870
34/10870
35/10870
36/10870
37/10870
38/10870
39/10870
40/10870
41/10870
42/10870
43/10870
44/10870
45/10870
46/10870
47/10870
48/10870
49/10870
50/10870
51/10870
52/10870
53/10870
54/10870
55/10870
56/10870
57/10870
58/10870
59/10870
60/10870
61/10870
62/10870
63/10870
64/10870
65/10870
66/10870
67/10870
68/10870
69/10870
70/10870
71/10870
72/10870
73/10870
74/10870
75/10870
76/10870
77/10870
78/10870
79/10870
80/10870
81/10870
82/10870
83/10870
84/10870
85/10870
86/10870
87/10870
88/10870
89/10870
90/10870
91/10870
92/10870
93/10870
94/10870
95/10870
96/10870
97/10870
98/10870
99/10870
100/10870
101/10870
102/10870
103/10870
104/10870
105/10870
106/10870
107/10870
108/10870
109/10870
110/10870


2718/10870
2719/10870
2720/10870
2721/10870
2722/10870
2723/10870
2724/10870
2725/10870
2726/10870
2727/10870
2728/10870
2729/10870
2730/10870
2731/10870
2732/10870
2733/10870
2734/10870
2735/10870
2736/10870
2737/10870
2738/10870
2739/10870
2740/10870
2741/10870
2742/10870
2743/10870
2744/10870
2745/10870
2746/10870
2747/10870
2748/10870
2749/10870
2750/10870
2751/10870
2752/10870
2753/10870
2754/10870
2755/10870
2756/10870
2757/10870
2758/10870
2759/10870
2760/10870
2761/10870
2762/10870
2763/10870
2764/10870
2765/10870
2766/10870
2767/10870
2768/10870
2769/10870
2770/10870
2771/10870
2772/10870
2773/10870
2774/10870
2775/10870
2776/10870
2777/10870
2778/10870
2779/10870
2780/10870
2781/10870
2782/10870
2783/10870
2784/10870
2785/10870
2786/10870
2787/10870
2788/10870
2789/10870
2790/10870
2791/10870
2792/10870
2793/10870
2794/10870
2795/10870
2796/10870
2797/10870
2798/10870
2799/10870
2800/10870
2801/10870
2802/10870
2803/10870
2804/10870
2805/10870
2806/10870
2807/10870
2808/10870

5304/10870
5305/10870
5306/10870
5307/10870
5308/10870
5309/10870
5310/10870
5311/10870
5312/10870
5313/10870
5314/10870
5315/10870
5316/10870
5317/10870
5318/10870
5319/10870
5320/10870
5321/10870
5322/10870
5323/10870
5324/10870
5325/10870
5326/10870
5327/10870
5328/10870
5329/10870
5330/10870
5331/10870
5332/10870
5333/10870
5334/10870
5335/10870
5336/10870
5337/10870
5338/10870
5339/10870
5340/10870
5341/10870
5342/10870
5343/10870
5344/10870
5345/10870
5346/10870
5347/10870
5348/10870
5349/10870
5350/10870
5351/10870
5352/10870
5353/10870
5354/10870
5355/10870
5356/10870
5357/10870
5358/10870
5359/10870
5360/10870
5361/10870
5362/10870
5363/10870
5364/10870
5365/10870
5366/10870
5367/10870
5368/10870
5369/10870
5370/10870
5371/10870
5372/10870
5373/10870
5374/10870
5375/10870
5376/10870
5377/10870
5378/10870
5379/10870
5380/10870
5381/10870
5382/10870
5383/10870
5384/10870
5385/10870
5386/10870
5387/10870
5388/10870
5389/10870
5390/10870
5391/10870
5392/10870
5393/10870
5394/10870

Failed to find class field-validation-error
7185/10870
Failed to find class field-validation-error
7186/10870
Failed to find class field-validation-error
7187/10870
Failed to find class field-validation-error
7188/10870
Failed to find class field-validation-error
7189/10870
Failed to find class field-validation-error
7190/10870
Failed to find class field-validation-error
7191/10870
Failed to find class field-validation-error
7192/10870
Failed to find class field-validation-error
7193/10870
Failed to find class field-validation-error
7194/10870
Failed to find class field-validation-error
7195/10870
Failed to find class field-validation-error
7196/10870
Failed to find class field-validation-error
7197/10870
Failed to find class field-validation-error
7198/10870
Failed to find class field-validation-error
7199/10870
Failed to find class field-validation-error
7200/10870
Failed to find class field-validation-error
7201/10870
Failed to find class field-validation-error
7202/10870
Failed to 

Failed to find class field-validation-error
7334/10870
Failed to find class field-validation-error
7335/10870
Failed to find class field-validation-error
7336/10870
Failed to find class field-validation-error
7337/10870
Failed to find class field-validation-error
7338/10870
Failed to find class field-validation-error
7339/10870
Failed to find class field-validation-error
7340/10870
Failed to find class field-validation-error
7341/10870
Failed to find class field-validation-error
7342/10870
Failed to find class field-validation-error
7343/10870
Failed to find class field-validation-error
7344/10870
Failed to find class field-validation-error
7345/10870
Failed to find class field-validation-error
7346/10870
Failed to find class field-validation-error
7347/10870
Failed to find class field-validation-error
7348/10870
Failed to find class field-validation-error
7349/10870
Failed to find class field-validation-error
7350/10870
Failed to find class field-validation-error
7351/10870
Failed to 

...

10162/10870
Failed to find class field-validation-error
10163/10870
Failed to find class field-validation-error
10164/10870
Failed to find class field-validation-error
10165/10870
Failed to find class field-validation-error
10166/10870
Failed to find class field-validation-error
10167/10870
Failed to find class field-validation-error
10168/10870
Failed to find class field-validation-error
10169/10870
Failed to find class field-validation-error
10170/10870
Failed to find class field-validation-error
10171/10870
Failed to find class field-validation-error
10172/10870
Failed to find class field-validation-error
10173/10870
Failed to find class field-validation-error
10174/10870
Failed to find class field-validation-error
10175/10870
Failed to find class field-validation-error
10176/10870
Failed to find class field-validation-error
10177/10870
Failed to find class field-validation-error
10178/10870
Failed to find class field-validation-error
10179/10870
Failed to find class field-validatio

10309/10870
Failed to find class field-validation-error
10310/10870
Failed to find class field-validation-error
10311/10870
Failed to find class field-validation-error
10312/10870
Failed to find class field-validation-error
10313/10870
Failed to find class field-validation-error
10314/10870
Failed to find class field-validation-error
10315/10870
Failed to find class field-validation-error
10316/10870
Failed to find class field-validation-error
10317/10870
Failed to find class field-validation-error
10318/10870
Failed to find class field-validation-error
10319/10870
Failed to find class field-validation-error
10320/10870
Failed to find class field-validation-error
10321/10870
Failed to find class field-validation-error
10322/10870
Failed to find class field-validation-error
10323/10870
Failed to find class field-validation-error
10324/10870
Failed to find class field-validation-error
10325/10870
Failed to find class field-validation-error
10326/10870
Failed to find class field-validatio

10456/10870
Failed to find class field-validation-error
10457/10870
Failed to find class field-validation-error
10458/10870
Failed to find class field-validation-error
10459/10870
Failed to find class field-validation-error
10460/10870
Failed to find class field-validation-error
10461/10870
Failed to find class field-validation-error
10462/10870
Failed to find class field-validation-error
10463/10870
Failed to find class field-validation-error
10464/10870
Failed to find class field-validation-error
10465/10870
Failed to find class field-validation-error
10466/10870
Failed to find class field-validation-error
10467/10870
Failed to find class field-validation-error
10468/10870
Failed to find class field-validation-error
10469/10870
Failed to find class field-validation-error
10470/10870
Failed to find class field-validation-error
10471/10870
Failed to find class field-validation-error
10472/10870
Failed to find class field-validation-error
10473/10870
Failed to find class field-validatio

10603/10870
Failed to find class field-validation-error
10604/10870
Failed to find class field-validation-error
10605/10870
Failed to find class field-validation-error
10606/10870
Failed to find class field-validation-error
10607/10870
Failed to find class field-validation-error
10608/10870
Failed to find class field-validation-error
10609/10870
Failed to find class field-validation-error
10610/10870
Failed to find class field-validation-error
10611/10870
Failed to find class field-validation-error
10612/10870
Failed to find class field-validation-error
10613/10870
Failed to find class field-validation-error
10614/10870
Failed to find class field-validation-error
10615/10870
Failed to find class field-validation-error
10616/10870
Failed to find class field-validation-error
10617/10870
Failed to find class field-validation-error
10618/10870
Failed to find class field-validation-error
10619/10870
Failed to find class field-validation-error
10620/10870
Failed to find class field-validatio

10750/10870
Failed to find class field-validation-error
10751/10870
Failed to find class field-validation-error
10752/10870
Failed to find class field-validation-error
10753/10870
Failed to find class field-validation-error
10754/10870
Failed to find class field-validation-error
10755/10870
Failed to find class field-validation-error
10756/10870
Failed to find class field-validation-error
10757/10870
Failed to find class field-validation-error
10758/10870
Failed to find class field-validation-error
10759/10870
Failed to find class field-validation-error
10760/10870
Failed to find class field-validation-error
10761/10870
Failed to find class field-validation-error
10762/10870
Failed to find class field-validation-error
10763/10870
Failed to find class field-validation-error
10764/10870
Failed to find class field-validation-error
10765/10870
Failed to find class field-validation-error
10766/10870
Failed to find class field-validation-error
10767/10870
Failed to find class field-validatio